**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [127]:
!pip install scikit-video
!pip install opencv-contrib-python

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io as skvideo
import cv2
import json
from pathlib import Path

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Within the act function, the agent chooses (with a probability of epsilon) a random action randomly at uniform. On the other hand, the agent selects an action based on a learned policy with a probability of 1-epsilon. This forms the underlying exploration-exploitation tradeoff that is fundamental to reinforcement learning. A small value for epsilon leads to behavior that favors exploiting the learned policy; if epsilon is too small, the agent can converge to a suboptimal policy by exploiting what it learns early on in the task and disregarding other policies that are potentially more optimal (this characterizes a greedy agent). In contrast, a high value for epsilon causes the agent to constantly explore, which reduces the cumulative reward accrued over time, which is the ultimate goal of a reinforcement learning agent. Therefore, a balance between exploration and exploitation is necessary in order to learn the optimal policy while also maximizing future reward. There are other more sophisticated techniques to the epsilon greedy (e.g. using a softmax).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array indicates the location of the agent within the gridworld (with an additional 2 cells on top of each border in order to accommodate for the 5 by 5 vision range of the agent). The current position is set to 1, with 0 at every toher location other than the ones where the agent cannot go, which are set to -1.

The board array is a grid of the same size as the position array, with zeros for all empty cells. It also describes the reward (0.5 for bonus cells and -1 for malus cells). The array board contains the informations concerning the cheese (cells containing 0.5) and the poisonous cells (cells containing -1). Hence, when the rat goes to a certain position, the value of board wrt that location is added to the reward. Subsequently, the value of the cell is set to 0 on the board, indicating that the agent has consumed either the food or the poison.

Both arrays are updated after each action; they are used to keep track of the experiment and also to compute the state.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # reset state at every epoch
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        #initialize
        win = 0
        lose = 0

        while not game_over:
            # take an action
            action = agent.act(state)

            # act in the environment with the above action
            # get the successor state, the reward from transition
            # and whether the game has been terminated
            prev_state = state
            state, reward, game_over = env.act(action)

            # update the win/lose counters based on the reward
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [135]:
# Initialize the game

import skvideo.io
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')

Win/lose count 11.0/21.0. Average score (-10.0)
Win/lose count 13.5/7.0. Average score (-1.75)
Win/lose count 12.5/19.0. Average score (-3.3333333333333335)
Win/lose count 17.0/20.0. Average score (-3.25)
Win/lose count 9.5/14.0. Average score (-3.5)
Win/lose count 16.0/14.0. Average score (-2.5833333333333335)
Win/lose count 11.5/14.0. Average score (-2.5714285714285716)
Win/lose count 15.0/20.0. Average score (-2.875)
Win/lose count 11.0/11.0. Average score (-2.5555555555555554)
Win/lose count 10.5/12.0. Average score (-2.45)
Win/lose count 10.0/13.0. Average score (-2.5)
Win/lose count 8.5/19.0. Average score (-3.1666666666666665)
Win/lose count 15.0/15.0. Average score (-2.923076923076923)
Win/lose count 12.0/12.0. Average score (-2.7142857142857144)
Win/lose count 14.0/18.0. Average score (-2.8)
Win/lose count 9.5/17.0. Average score (-3.09375)
Win/lose count 7.5/17.0. Average score (-3.4705882352941178)
Win/lose count 12.5/15.0. Average score (-3.4166666666666665)
Win/lose count 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}





First, it must be shown that $Q^\pi$ satisfies the Bellman equation:
$$\begin{align}
Q^\pi(s,a) = E_{p^{\pi}}\left[\sum_{t \ge 0}\gamma^{t}r(s_{t},a_{t}) ~\middle|~ s_{0}=s,a_{0}=a\right] \\
= E_{p^{\pi}}\left[r(s_0,a_0) + \gamma \sum_{t \ge 1}\gamma^{t-1}r(s_{t},a_{t}) ~\middle|~ s_{0}=s,a_{0}=a\right] \\
= E_{(s',a')\sim p^\pi(.|s,a)}\left[r(s, a) + \gamma E_{p^{\pi}} \left[ \sum_{t \ge 0}\gamma^{t}r(s_{t-1},a_{t-1}) ~\middle|~ s_1=s', a_1=a' \right] ~\middle|~ s_{0}=s,a_{0}=a\right] \\
= E_{(s',a')\sim p^\pi(.|s,a)}\left[r(s, a) + \gamma Q^\pi (s', a')\right]
\end{align}$$
Now, the proof for the optimal Bellman equation must be demonstrated. Within the context of an MDP, there exists an optimal policy that is deterministic. Therefore, given a deterministic optimal policy ($\pi^*$), it is true that $Q^*=Q^{\pi^*}$. For each state ($s$), there exists an action ($a$), such that $\pi^*(a|s) = 1$, and $Q^{\pi^*}(s,a)=\max_{a'} Q^{\pi^*}(s, a')$ (since $\pi^*$ would otherwise not be the optimal policy). Given a deterministic policy, the policy function can be denoited $\pi^*(s)$. It follows that:
$$\begin{align}
Q^*(s,a) = Q^{\pi^*}(s,a) = E_{s' \sim p(.|s,a)}\left[r(s, a) + \gamma Q^{\pi^*} (s', \pi^*(s'))\right] \\
= E_{s' \sim p(.|s,a)}\left[r(s, a) + \gamma \max_{a'} Q^{\pi^*} (s', a')\right] \\
= E_{s' \sim p(.|s,a)}\left[r(s, a) + \gamma \max_{a'} Q^* (s', a')\right]
\end{align}$$
The final inequality can be re-written as follows: $E_{s' \sim p(.|s,a)}\left[r(s, a) + \gamma \max_{a'} Q^* (s', a') - Q^*(s,a)\right] = 0$. Since $r(s, a) + \gamma \max_{a'} Q^* (s', a')$ should ultimately equal $Q^*(s,a)$, it makes sense to minimize the norm of this quantity, such that the objective function would be the following: $\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$. This is because the loss function would satisfy the following: $\mathcal{L}(\theta) \geq 0$ and $\mathcal{L}(\theta^*) = 0$. Furthermore, this function is minimized by $\theta^*$ associated to the optimal Q function.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        [n,m,l] = np.shape(s)
        state = np.reshape(s,(1,n,m,l))
        pred = self.model.predict(state)
        return np.argmax(pred)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            state, next_state, action, reward, game_over = self.memory.random_access()
            input_states[i] = state
            [n,m,l] = np.shape(state)
            state = np.reshape(state,(1,n,m,l))
            target_q[i] = self.model.predict(state)
            if game_over_:
                ######## FILL IN
                target_q[i, action] = reward
            else:
                ######## FILL IN
                [n,m,l] = np.shape(next_state)
                next_state = np.reshape(next_state,(1,n,m,l))
                target_q[i, action] = reward + self.discount*np.max(self.model.predict(next_state))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(40,activation ='relu', input_shape=(5,5,self.n_state)))
        model.add(Dense(20,activation ='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [139]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0141 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/050 | Loss 0.0192 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/050 | Loss 0.0572 | Win/lose count 4.0/1.0 (3.0)
Epoch 003/050 | Loss 0.1906 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/050 | Loss 0.5843 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/050 | Loss 1.0210 | Win/lose count 2.0/3.0 (-1.0)
Epoch 006/050 | Loss 1.7368 | Win/lose count 7.0/2.0 (5.0)
Epoch 007/050 | Loss 1.7872 | Win/lose count 7.5/3.0 (4.5)
Epoch 008/050 | Loss 1.6845 | Win/lose count 7.0/1.0 (6.0)
Epoch 009/050 | Loss 1.8877 | Win/lose count 6.0/3.0 (3.0)
Epoch 010/050 | Loss 1.8922 | Win/lose count 1.5/4.0 (-2.5)
Epoch 011/050 | Loss 1.7934 | Win/lose count 2.0/6.0 (-4.0)
Epoch 012/050 | Loss 1.8005 | Win/lose count 18.0/4.0 (14.0)
Epoch 013/050 | Loss 2.0383 | Win/lose count 11.0/5.0 (6.0)
Epoch 014/050 | Loss 1.9517 | Win/lose count 10.5/3.0 (7.5)
Epoch 015/050 | Loss 1.8981 | Win/lose count 5.0/6.0 (-1.0)
Epoch 016/050 | Loss 1.8184 | Win/lose count 1

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN

        input_shape = (5, 5, self.n_state)
        model = Sequential()
        model.add(Conv2D(32,(2,2),input_shape=input_shape,activation='relu'))
        model.add(Conv2D(16,(2,2),activation='relu'))
        model.add(Reshape((-1,)))
        model.add(Dense(4))
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [141]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 3, 3, 16)          2064      
_________________________________________________________________
reshape_9 (Reshape)          (None, 144)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 4)                 580       
Total params: 2,932
Trainable params: 2,932
Non-trainable params: 0
_________________________________________________________________
Epoch 000/050 | Loss 0.0019 | Win/lose count 0.5/5.0 (-4.5)
Epoch 001/050 | Loss 0.0088 | Win/lose count 3.0/7.0 (-4.0)
Epoch 002/050 | Loss 0.0084 | Win/lose count 2.0/3.0 (-1.0)
Epoch 003/050 | Loss 0.0120 | Win/lose count 1.0/1.0 (0.0)
Epoch 004/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [142]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 4, 4, 32)          288       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 3, 3, 16)          2064      
_________________________________________________________________
reshape_10 (Reshape)         (None, 144)               0         
_________________________________________________________________
dense_67 (Dense)             (None, 4)                 580       
Total params: 2,932
Trainable params: 2,932
Non-trainable params: 0
_________________________________________________________________
Test of the CNN
Win/lose count 5.0/5.0. Average score (0.0)
Win/lose count 4.5/0. Average score (2.25)
Win/lose count 2.5/0. Average score (2.3333333333333335)
Win/lose count 2.5/2.0. Average score (1.875)
Win/lose count 4.5/1.0. Average score (2.2)

The CNN architecture achieved better performance than the FC model, which was even more pronounced when increasing the temperature hyper-parameter.

A major issue for both models is that they have limited exploration (due to having a low value for epsilon). The agent seems to get consistently transitioning between two states. This is due to fact that the agent does not retain information about having already visited a state.

Higher values for the temperature hyper-parameter leads to better performance for both models. This can be attributed to the fact that there are several positively rewarding states, so at beginning of the run, the agent will likely be surrounded by positive rewards in the beginning and the agent will just exploit by remaining in the area due to the low exploratory behavior. On the other hand, lower values for the temperature hyper-parameter leads both models having negative scores.

In [149]:
HTML(display_videos('cnn_test10.mp4'))

In [144]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # We decrease epsilon
        agent.set_epsilon(agent.epsilon*0.9)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        # Array to indicate if a cell has been explored
        self.malus_position = np.zeros((grid_size, grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2 :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [146]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 4, 4, 32)          416       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 3, 3, 16)          2064      
_________________________________________________________________
reshape_11 (Reshape)         (None, 144)               0         
_________________________________________________________________
dense_71 (Dense)             (None, 4)                 580       
Total params: 3,060
Trainable params: 3,060
Non-trainable params: 0
_________________________________________________________________
Epoch 000/050 | Loss 0.0069 | Win/lose count 4.0/20.00000000000002 (-16.00000000000002)
Epoch 001/050 | Loss 0.0022 | Win/lose count 1.0/23.30000000000007 (-22.30000000000007)
Epoch 002/050 | Loss 0.0145 | Win/lose count 4.0/18.9 (-14.89999999999999

In [147]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 19.5/2.0. Average score (17.5)
Win/lose count 21.0/4.0. Average score (17.25)
Win/lose count 22.0/4.0. Average score (17.5)
Win/lose count 20.5/4.0. Average score (17.25)
Win/lose count 23.0/2.0. Average score (18.0)
Win/lose count 6.5/0. Average score (16.083333333333332)
Win/lose count 22.0/1.0. Average score (16.785714285714285)
Win/lose count 26.0/4.0. Average score (17.4375)
Win/lose count 25.0/3.0. Average score (17.944444444444443)
Win/lose count 23.0/5.0. Average score (17.95)
Win/lose count 18.0/3.0. Average score (17.681818181818183)
Win/lose count 16.5/2.0. Average score (17.416666666666668)
Win/lose count 24.5/7.0. Average score (17.423076923076923)
Win/lose count 22.0/4.0. Average score (17.464285714285715)
Win/lose count 28.5/2.0. Average score (18.066666666666666)
Win/lose count 24.5/3.0. Average score (18.28125)
Win/lose count 18.0/4.0. Average score (18.029411764705884)
Win/lose count 21.0/4.0. Average score (17.97222222222222)
Win/lose count 17.0/0. Ave

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***